In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from datetime import timedelta
import time
import datetime
import os

# Préparation des inputs

In [22]:
# Paramètre : Regroupement des données pour les input
step_group = "60min"

Chargement des données

In [23]:
data = pd.read_csv(r'C:\Users\quent\Documents\IMTA3\PO\V0\Cleaning\data_cleaned.csv')
del data['Unnamed: 0']

Passage des dates au format Timestamp

In [24]:
data['date'] = data['date'].apply(lambda x : pd.Timestamp(x))

One hot encoding pour la variable CBN

In [25]:
data['CBN'] = pd.Categorical(data['CBN'])

# Créer les one hot encodings pour chaque variabble
dfDummies_cbn = pd.get_dummies(data['CBN'], prefix = 'CBN')

# Concatener les one hot encodings avec les données
data = pd.concat([data, dfDummies_cbn], axis=1)

# Suppression de la variable CBBN
data.drop(columns=['CBN'], inplace=True)

# On ordonne les data par "date"
data.sort_values(by='date', inplace=True)

# Réindexage des lignes
data.reset_index(inplace=True)
data.drop(columns=['index'], inplace=True)

In [26]:
data.head()

,date,CBN_0 02,CBN_0 03,CBN_0 04,CBN_0 05,CBN_0 06,CBN_0 07,CBN_0 08,CBN_0 09,CBN_0 10,...,CBN_X 06,CBN_X 07,CBN_X 08,CBN_X 09,CBN_X 10,CBN_X 11,CBN_X 12,CBN_Y,CBN_Z,CBN_j 04
0,2018-07-21 04:52:46,0,0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,2018-07-21 04:52:49,0,0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,2018-07-21 04:52:57,0,0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,2018-07-21 04:53:01,0,0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,2018-07-21 04:53:04,0,0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


Formatage des input

In [27]:
start = time.time()

grouped_data = data.groupby(pd.Grouper(key="date", freq=step_group)).sum()

print("Temps de création : {:.2f} s".format(time.time() - start))

# Ajout d'un index
grouped_data.reset_index(inplace=True)

Temps de création : 266.51 s


Ajout variable mois et jours de la semaine

In [28]:
grouped_data['month'] = grouped_data['date'].apply(lambda x : x.month)
grouped_data['weekday'] = grouped_data['date'].apply(lambda x : x.weekday())

One hot encoding pour les varialbes jours de la semaine et mois

In [29]:
grouped_data['month'] = pd.Categorical(grouped_data['month'])
grouped_data['weekday'] = pd.Categorical(grouped_data['weekday'])

# Créer les one hot encodings pour chaque variabble
dfDummies_month = pd.get_dummies(grouped_data['month'], prefix = 'month')
dfDummies_weekday = pd.get_dummies(grouped_data['weekday'], prefix = 'weekday')

# Concatener les one hot encodings avec les données
grouped_data = pd.concat([grouped_data, dfDummies_weekday, dfDummies_month], axis=1)

# Suppression de la variable mois et jour de la semaine
grouped_data.drop(columns=['month', 'weekday'], inplace=True)

In [30]:
grouped_data.head()

,date,CBN_0 02,CBN_0 03,CBN_0 04,CBN_0 05,CBN_0 06,CBN_0 07,CBN_0 08,CBN_0 09,CBN_0 10,...,month_3,month_4,month_5,month_6,month_7,month_8,month_9,month_10,month_11,month_12
0,2018-07-21 04:00:00,0,0,0,31,0,0,0,0,0,...,0,0,0,0,1,0,0,0,0,0
1,2018-07-21 05:00:00,0,0,0,0,0,16,0,0,0,...,0,0,0,0,1,0,0,0,0,0
2,2018-07-21 06:00:00,0,0,0,0,0,6,0,0,0,...,0,0,0,0,1,0,0,0,0,0
3,2018-07-21 07:00:00,0,1,0,7,0,0,0,0,0,...,0,0,0,0,1,0,0,0,0,0
4,2018-07-21 08:00:00,0,0,0,24,0,1,0,0,0,...,0,0,0,0,1,0,0,0,0,0


Sauvegarde des données en pickel

In [31]:
grouped_data.to_pickle(r"C:\Users\quent\Documents\IMTA3\PO\V2\Inputs\group{}_inputs.pkl".format(step_group))